<a href="https://colab.research.google.com/github/kovzanok/dls-final-task/blob/main/Final_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Установка зависимостей

In [ ]:
pip install insightface

In [ ]:
from torch import nn
from insightface.app import FaceAnalysis
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import torchvision.transforms.functional as TF
import cv2

import random
import os

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Подготовка данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Архив изображений

In [ ]:
!unzip -q /content/drive/MyDrive/img_align_celeba.zip -d /content/celeba/

##Разметка "имя изображения  - id"

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/identity_CelebA.txt', delim_whitespace=True, header=None,index_col=0)

In [ ]:
df.index.name = 'image_name'
df.rename(columns={1:'id'},inplace=True)

In [ ]:
def get_filenames_os(folder_path):
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    return files

In [ ]:
filenames = get_filenames_os('/content/content/stage_3_dataset')
filtered_df = df.loc[filenames]
le = LabelEncoder()
filtered_df['encoded_id'] = le.fit_transform(filtered_df['id'])

#Пайплайн